# Import

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
save_path = "../../interactive-2.0/WR/the-525-rule"

# Load Data

In [ ]:
df = pd.read_pickle("../../FantasyData/data-frames/df_basic_ngs_1999_2023.pkl")

# Prep Data

In [ ]:
df_wr = df.copy().query("position == 'WR'")

In [ ]:
df_wr.dropna(subset="Draft Year", inplace=True)

In [ ]:
df_wr['Draft Year'] = df_wr['Draft Year'].astype(str).str.replace('s', '')
df_wr["Draft Year"] = df_wr["Draft Year"].astype(int)

In [ ]:
df_wr.head()

# Create Rook Year DF

In [ ]:
df_rook_year = df_wr.copy().loc[df_wr["season"] == df_wr["Draft Year"]]

In [ ]:
df_rook_year.query("player_name == 'Sammy Watkins'")[["receiving_yards"]]

In [ ]:
fig = px.scatter(df_rook_year, x='Draft Year', y='receiving_yards', hover_data=["player_name"])

# Update layout for clarity.
fig.update_layout(xaxis_title='Draft Year', yaxis_title='Receiving Yards')

fig.add_hline(y=525, line_dash="dash", line_color="red")

# Show the figure.
# fig.write_html(f"{save_path}/0-1000-bubble.html")
fig.show()

# Yard Bucket by Mode

In [ ]:
# Calculate the mode of the 'tier' for each player
mode_tiers = df_wr.groupby('player_id')['tier'].agg(lambda x: pd.Series.mode(x).iloc[0]).reset_index()
mode_tiers.rename(columns={'tier': 'Min Mode Tier'}, inplace=True)

# Merge the mode tiers back into the rookie year DataFrame
df_rook_year_update = df_wr[df_wr["season"] == df_wr["Draft Year"]].merge(mode_tiers, on='player_id', how='left')

# Define yardage categories with accurate labels
bins = [0, 100, 200, 300, 400, 500, 600, 700, 800, float('inf')]
labels = ['0-99', '100-199', '200-299', '300-399', '400-499', '500-599', '600-699', '700-799', '800+']

# Categorize the receiving yards
df_rook_year_update['Yards as Rookie'] = pd.cut(df_rook_year_update['receiving_yards'], bins=bins, labels=labels, right=False)

# Group by 'yard_category' and 'mode_tier', then count the occurrences
result = df_rook_year_update.groupby(['Yards as Rookie', 'Min Mode Tier']).size().unstack(fill_value=0)

# Display the result
result

Color options for the heatmap:
```
'aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
 'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
 'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
 'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
 'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
 'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
 'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
 'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
 'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
 'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
 'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
 'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
 'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
 'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
 'ylorrd'
 ```

In [ ]:
import plotly.graph_objects as go

# Use your actual 'result' DataFrame from the previous step where you calculated the counts.
# Replace 'result' below with your actual DataFrame variable.

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=result.values,  # The counts from your DataFrame
    x=result.columns,  # These are the Min Mode Tier values
    y=result.index,  # These are the Yards as Rookie categories
    hoverongaps=False,
    colorscale='blackbody'
))

# Update the layout for a better presentation
fig.update_layout(
    title='Heatmap of Rookie Yard Tiers vs. Mode Tiers',
    xaxis_title='Min Mode Tier',
    yaxis_title='Yards as Rookie',
    xaxis_nticks=36  # Adjust this if you need more or fewer ticks based on your actual tiers
)

# Show the figure
fig.show()

In [ ]:
result.iloc[1:]

In [ ]:
result.to_html(f"{save_path}/bins-800.html")

In [ ]:
import plotly.graph_objects as go

# Use your actual 'result' DataFrame from the previous step where you calculated the counts.
# Replace 'result' below with your actual DataFrame variable.

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=result.iloc[1:].values,  # The counts from your DataFrame
    x=result.iloc[1:].columns,  # These are the Min Mode Tier values
    y=result.iloc[1:].index,  # These are the Yards as Rookie categories
    hoverongaps=False,
    colorscale='blackbody'
))

# Update the layout for a better presentation
fig.update_layout(
    title='Heatmap of Rookie Yard Tiers vs. Mode Tiers',
    xaxis_title='Min Mode Tier',
    yaxis_title='Yards as Rookie',
    xaxis_nticks=36  # Adjust this if you need more or fewer ticks based on your actual tiers
)

# Show the figure
fig.show()

In [ ]:
row_totals = result.sum(axis=1)
result_percentage_by_row = round(result.div(row_totals, axis=0) * 100, 2)
result_percentage_by_row

In [ ]:
import plotly.graph_objects as go

# Use your actual 'result' DataFrame from the previous step where you calculated the counts.
# Replace 'result' below with your actual DataFrame variable.

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=result_percentage_by_row.iloc[0:].values,  # The counts from your DataFrame
    x=result_percentage_by_row.iloc[0:].columns,  # These are the Min Mode Tier values
    y=result_percentage_by_row.iloc[0:].index,  # These are the Yards as Rookie categories
    hoverongaps=False,
    colorscale='blackbody'
))

# Update the layout for a better presentation
fig.update_layout(
    title='Heatmap of Rookie Yard Tiers vs. Min Mode Tiers Percentages',
    xaxis_title='Min Mode Tier',
    yaxis_title='Yards as Rookie',
    xaxis_nticks=36  # Adjust this if you need more or fewer ticks based on your actual tiers
)

# Show the figure
fig.write_html(f"{save_path}/heat-map-800-bins-percentage.html")
fig.show()

In [ ]:
import plotly.graph_objects as go

# Use your actual 'result' DataFrame from the previous step where you calculated the counts.
# Replace 'result' below with your actual DataFrame variable.

min_modes_to_graph = [1.0, 2.0, 3.0, 4.0, 5.0]

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=result_percentage_by_row.iloc[3:][min_modes_to_graph].values,  # The counts from your DataFrame
    x=result_percentage_by_row.iloc[3:][min_modes_to_graph].columns,  # These are the Min Mode Tier values
    y=result_percentage_by_row.iloc[3:][min_modes_to_graph].index,  # These are the Yards as Rookie categories
    hoverongaps=False,
    colorscale='blackbody'
))

# Update the layout for a better presentation
fig.update_layout(
    title='Heatmap of Rookie Yard Tiers vs. Min Mode Tiers Percentages Zoomed In',
    xaxis_title='Min Mode Tier',
    yaxis_title='Yards as Rookie',
    xaxis_nticks=8  # Adjust this if you need more or fewer ticks based on your actual tiers
)

# Show the figure
fig.write_html(f"{save_path}/heat-map-800-bins-percentage-zoomed.html")
fig.show()

In [ ]:
result_percentage_by_row.to_html(f"{save_path}/bins-800-percentage.html")

In [ ]:
# Calculate the mode of the 'tier' for each player
mode_tiers = df_wr.groupby('player_id')['tier'].agg(lambda x: pd.Series.mode(x).iloc[0]).reset_index()
mode_tiers.rename(columns={'tier': 'Min Mode Tier'}, inplace=True)

# Merge the mode tiers back into the rookie year DataFrame
df_rook_year_update = df_wr[df_wr["season"] == df_wr["Draft Year"]].merge(mode_tiers, on='player_id', how='left')

# Define yardage categories with accurate labels
bins = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, float('inf')]
labels = ['0-99', '100-199', '200-299', '300-399', '400-499', '500-599', '600-699', '700-799', 
          '800-899', '900-999', '1000+']

# Categorize the receiving yards
df_rook_year_update['Yards as Rookie'] = pd.cut(df_rook_year_update['receiving_yards'], bins=bins, labels=labels, right=False)

# Group by 'yard_category' and 'mode_tier', then count the occurrences
result = df_rook_year_update.groupby(['Yards as Rookie', 'Min Mode Tier']).size().unstack(fill_value=0)

# Display the result
result

In [ ]:
result = result.iloc[5:]

In [ ]:
result.to_html(f"{save_path}/elites.html")

In [ ]:
result

### Percentages by Row

In [ ]:
row_totals = result.sum(axis=1)
result_percentage_by_row = round(result.div(row_totals, axis=0) * 100, 2)

In [ ]:
result_percentage_by_row

In [ ]:
# result_percentage_by_row.to_html(f"{save_path}/elites-percentages.html")

In [ ]:
df_graph = result_percentage_by_row.copy()
df_graph.reset_index(inplace=True)
df_graph["min_mode_tier"] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
df_graph

In [ ]:
df_melted = df_graph.melt(id_vars=['min_mode_tier', 'Yards as Rookie'], var_name='Min Mode Tier', value_name='Value')
fig = px.scatter(df_melted, x='Min Mode Tier', y='Value', size='Value', color='Yards as Rookie',
                 category_orders={"Yards as Rookie": ['0-99', '100-199', '200-299', '300-399', '400-499',
                                                      '500-599', '600-699', '700-799', 
          '800-899', '900-999', '1000+']},
                 labels={'Value': 'Percentage (%)'},
                 title='Bubble Chart of Percentages by Min Mode Tier and Yards Category',
                 size_max=60)

# Update layout for clarity.
fig.update_layout(xaxis_title='Min Mode Tier', yaxis_title='Percentage (%)')

# Show the figure.
fig.write_html(f"{save_path}/0-1000-bubble.html")
fig.show()

In [ ]:
# Assuming your DataFrame is named 'result_percentage_by_row'
tier_1_data = result_percentage_by_row[list(result_percentage_by_row.columns[:5])]  # Selects the data for 'tier 1'
tier_1_data

In [ ]:
tier_1_data.to_html(f"{save_path}/elites-percentages-zoomed.html")

# The Great Filter

### 575 Rule

In [ ]:
# Calculate the mode of the 'tier' for each player
mode_tiers = df_wr.groupby('player_id')['tier'].agg(lambda x: pd.Series.mode(x).iloc[0]).reset_index()
mode_tiers.rename(columns={'tier': 'Min Mode Tier'}, inplace=True)

# Merge the mode tiers back into the rookie year DataFrame
df_rook_year_update = df_wr[df_wr["season"] == df_wr["Draft Year"]].merge(mode_tiers, on='player_id', how='left')

# Define yardage categories with accurate labels
bins = [0, 575, float('inf')]
labels = ['0-574', '575+']

# Categorize the receiving yards
df_rook_year_update['Yards as Rookie'] = pd.cut(df_rook_year_update['receiving_yards'], bins=bins, labels=labels, right=False)

# Group by 'yard_category' and 'mode_tier', then count the occurrences
result = df_rook_year_update.groupby(['Yards as Rookie', 'Min Mode Tier']).size().unstack(fill_value=0)

# Display the result
result

In [ ]:
result.to_html(f"{save_path}/575-rule.html")

In [ ]:
row_totals = result.sum(axis=1)
result_percentage_by_row = round(result.div(row_totals, axis=0) * 100, 2)
result_percentage_by_row

In [ ]:
result_percentage_by_row.to_html(f"{save_path}/575-rule-percentage.html")

### 525 Rule

In [ ]:
# Calculate the mode of the 'tier' for each player
mode_tiers = df_wr.groupby('player_id')['tier'].agg(lambda x: pd.Series.mode(x).iloc[0]).reset_index()
mode_tiers.rename(columns={'tier': 'Min Mode Tier'}, inplace=True)

# Merge the mode tiers back into the rookie year DataFrame
df_rook_year_update = df_wr[df_wr["season"] == df_wr["Draft Year"]].merge(mode_tiers, on='player_id', how='left')

# Define yardage categories with accurate labels
bins = [0, 525, float('inf')]
labels = ['0-524', '525+']

# Categorize the receiving yards
df_rook_year_update['Yards as Rookie'] = pd.cut(df_rook_year_update['receiving_yards'], bins=bins, labels=labels, right=False)

# Group by 'yard_category' and 'mode_tier', then count the occurrences
result = df_rook_year_update.groupby(['Yards as Rookie', 'Min Mode Tier']).size().unstack(fill_value=0)

# Display the result
result

In [ ]:
result.to_html(f"{save_path}/525-rule.html")

In [ ]:
row_totals = result.sum(axis=1)
result_percentage_by_row = round(result.div(row_totals, axis=0) * 100, 2)
result_percentage_by_row

In [ ]:
result_percentage_by_row.to_html(f"{save_path}/525-rule-percentage.html")

In [ ]:
import plotly.graph_objects as go

# Use your actual 'result' DataFrame from the previous step where you calculated the counts.
# Replace 'result' below with your actual DataFrame variable.

df_graph = result_percentage_by_row.copy()

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=df_graph.values,  # The counts from your DataFrame
    x=df_graph.columns,  # These are the Min Mode Tier values
    y=df_graph.index,  # These are the Yards as Rookie categories
    hoverongaps=False,
    colorscale='blackbody'
))

# Update the layout for a better presentation
fig.update_layout(
    title='Heatmap of Rookie Yard Tiers vs. Mode Tiers',
    xaxis_title='Min Mode Tier',
    yaxis_title='Yards as Rookie',
    xaxis_nticks=36  # Adjust this if you need more or fewer ticks based on your actual tiers
)

# Show the figure
fig.show()

In [ ]:
df_graph

In [ ]:
df_graph.reset_index(inplace=True)

In [ ]:
df_graph["min_mode_tier"] = [1.0, 2.0]

In [ ]:
df_melted = df_graph.melt(id_vars=['min_mode_tier', 'Yards as Rookie'], var_name='Min Mode Tier', value_name='Value')

In [ ]:
df_melted.head()

In [ ]:
fig = px.bar(df_melted, x='Min Mode Tier', y='Value', color='Yards as Rookie', barmode='group',
             category_orders={"Yards as Rookie": ["0-524", "525+"]},
             labels={'Value': 'Percentage (%)'},
             title='Bar Chart of Percentages by Mode Tier and Yards Category')

# Update layout for clarity.
fig.update_layout(xaxis_title='Min Mode Tier', yaxis_title='Percentage (%)')

# Show the figure.
fig.show()

### 500 Rule

In [ ]:
# Calculate the mode of the 'tier' for each player
mode_tiers = df_wr.groupby('player_id')['tier'].agg(lambda x: pd.Series.mode(x).iloc[0]).reset_index()
mode_tiers.rename(columns={'tier': 'Min Mode Tier'}, inplace=True)

# Merge the mode tiers back into the rookie year DataFrame
df_rook_year_update = df_wr[df_wr["season"] == df_wr["Draft Year"]].merge(mode_tiers, on='player_id', how='left')

# Define yardage categories with accurate labels
bins = [0, 500, float('inf')]
labels = ['0-499', '500+']

# Categorize the receiving yards
df_rook_year_update['Yards as Rookie'] = pd.cut(df_rook_year_update['receiving_yards'], bins=bins, labels=labels, right=False)

# Group by 'yard_category' and 'mode_tier', then count the occurrences
result = df_rook_year_update.groupby(['Yards as Rookie', 'Min Mode Tier']).size().unstack(fill_value=0)

# Display the result
result

In [ ]:
result.to_html(f"{save_path}/500-rule.html")

In [ ]:
row_totals = result.sum(axis=1)
result_percentage_by_row = round(result.div(row_totals, axis=0) * 100, 2)
result_percentage_by_row

In [ ]:
result_percentage_by_row.to_html(f"{save_path}/500-rule-percentage.html")

In [ ]:
df_graph = result_percentage_by_row.copy()
df_graph.reset_index(inplace=True)
df_graph["min_mode_tier"] = [1.0, 2.0]
df_melted = df_graph.melt(id_vars=['min_mode_tier', 'Yards as Rookie'], var_name='Min Mode Tier', value_name='Value')
fig = px.scatter(df_melted, x='Min Mode Tier', y='Value', size='Value', color='Yards as Rookie',
                 category_orders={"Yards as Rookie": ["0-499", "500+"]},
                 labels={'Value': 'Percentage (%)'},
                 title='Bubble Chart of Percentages by Min Mode Tier and Yards Category',
                 size_max=60)

# Update layout for clarity.
fig.update_layout(xaxis_title='Min Mode Tier', yaxis_title='Percentage (%)')

# Show the figure.
fig.write_html(f"{save_path}/500-rule-bubble.html")
fig.show()

### Lowest Threshold

In [ ]:
# Calculate the mode of the 'tier' for each player
mode_tiers = df_wr.groupby('player_id')['tier'].agg(lambda x: pd.Series.mode(x).iloc[0]).reset_index()
mode_tiers.rename(columns={'tier': 'Min Mode Tier'}, inplace=True)

# Merge the mode tiers back into the rookie year DataFrame
df_rook_year_update = df_wr[df_wr["season"] == df_wr["Draft Year"]].merge(mode_tiers, on='player_id', how='left')

# Define yardage categories with accurate labels
bins = [0, 475, float('inf')]
labels = ['0-474', '475+']

# Categorize the receiving yards
df_rook_year_update['Yards as Rookie'] = pd.cut(df_rook_year_update['receiving_yards'], bins=bins, labels=labels, right=False)

# Group by 'yard_category' and 'mode_tier', then count the occurrences
result = df_rook_year_update.groupby(['Yards as Rookie', 'Min Mode Tier']).size().unstack(fill_value=0)

row_totals = result.sum(axis=1)
result_percentage_by_row = round(result.div(row_totals, axis=0) * 100, 2)
result_percentage_by_row

# Filters on Rook Year

In [ ]:
df_rook_year.query("receiving_yards >= 800")[["Draft Year", "player_name", "receiving_yards"]]

In [ ]:
df_filter = df_rook_year \
    .query("receiving_yards >= 1000") \
    .reset_index()[["Draft Year", "player_name", "receiving_yards"]]

In [ ]:
def highlight_row(row):
    return ['background-color: red' if row['Player Name'] == 'Michael Clayton' else '' for _ in row]

df_filter["receiving_yards"] = df_filter["receiving_yards"].astype(int)
df_filter.rename(columns={"player_name": "Player Name", "receiving_yards": "Yards as Rookie"}, inplace=True)
# Assuming 'df' is your DataFrame
df_highlighted = df_filter.style.apply(highlight_row, axis=1)

In [ ]:
df_highlighted = df_highlighted.set_properties(**{'border': '1px solid black'})

In [ ]:
df_highlighted

In [ ]:
df_highlighted.to_html(f"{save_path}/over-1000-yards-rookie-year.html")